<a href="https://colab.research.google.com/github/ShashankRaoM/app/blob/master/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#To download from google drive

download= drive.CreateFile({'id':'1yBKz5iYlvCeKBEZ4np4WXOrkmZYGQsDp'})
download.GetContentFile('X_test.hdf5')

download= drive.CreateFile({'id':'1HBZzwp2n1bt9IBA95pJlWjvLJGcpSq_s'})
download.GetContentFile('X_train.hdf5')

download= drive.CreateFile({'id':'13gL1vSCxb1r1QbJGRshkoEVhmIGxMJyx'})
download.GetContentFile('X_val.hdf5')

download= drive.CreateFile({'id':'1XY5hA_HwL1pqOzp-F6zm628bWSS9XeA8'})
download.GetContentFile('y_test.hdf5')

download= drive.CreateFile({'id':'14Sgpl6EqvUFncfyfei07kND1GKPClMWB'})
download.GetContentFile('y_train.hdf5')

download= drive.CreateFile({'id':'1k55de4jdMXlKD4U3wC3MnQzgWo9-Z974'})
download.GetContentFile('y_val.hdf5')

!pip install h5py
import h5py
with h5py.File('X_test.hdf5', 'r') as f:
   X_test = f['default'][:]

with h5py.File('X_train.hdf5', 'r') as f:
   X_train = f['default'][:]
    
with h5py.File('X_val.hdf5', 'r') as f:
   X_val = f['default'][:]

with h5py.File('y_test.hdf5', 'r') as f:
   y_test = f['default'][:]

with h5py.File('y_train.hdf5', 'r') as f:
   y_train = f['default'][:]
    
with h5py.File('y_val.hdf5', 'r') as f:
   y_val = f['default'][:]

    100% |████████████████████████████████| 993kB 19.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [2]:
import numpy as np
batch=X_test.shape[0]
frame=X_test.shape[1]
height=X_test.shape[2]
width=X_test.shape[3]
X_test=X_test.reshape(batch,frame,height,width,1)

batch=X_train.shape[0]
frame=X_train.shape[1]
height=X_train.shape[2]
width=X_train.shape[3]
X_train=X_train.reshape(batch,frame,height,width,1)
print(X_train.shape)

batch=X_val.shape[0]
frame=X_val.shape[1]
height=X_val.shape[2]
width=X_val.shape[3]
X_val=X_val.reshape(batch,frame,height,width,1)

(193118, 9, 30, 60, 1)


In [4]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers import Convolution3D, MaxPooling3D, ZeroPadding3D, BatchNormalization, LSTM, ConvLSTM2D,Activation,MaxPooling2D
from keras.optimizers import SGD

model = Sequential()
num_classes = 199
# 1st layer group
model.add(Convolution3D(128, 3, 3, 3, activation='relu', 
                        border_mode='same', name='conv1',
                        input_shape = (9,30,60,1),
                        dim_ordering='tf',
                        subsample=(1, 1, 1)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), 
                       border_mode='valid', name='pool1'))




# 2nd layer group
model.add(Convolution3D(256, 3, 3, 3, activation='relu', 
                        border_mode='same', name='conv2',
                        dim_ordering='tf',
                        subsample=(1, 1, 1)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                       border_mode='valid', name='pool2'))





# 3rd layer group
model.add(Convolution3D(512, 3, 3, 3, activation='relu', 
                        border_mode='same',
                        dim_ordering='tf',
                        name='conv3a',
                        subsample=(1, 1, 1)))
model.add(Convolution3D(512, 3, 3, 3, activation='relu', 
                        border_mode='same', 
                        dim_ordering='tf',
                        name='conv3b',
                        subsample=(1, 1, 1)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                       border_mode='valid', name='pool3'))

#LSTM

model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
               padding='same', return_sequences=True, name = 'LSTM1'))

model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
               padding='same', return_sequences=False, name = 'LSTM2'))

#spp
#model.add(Flatten())
#model.add(MaxPooling2D(pool_size=(28,28), strides=(28,28), border_mode='valid', name='spp1'))


#model.add(MaxPooling2D(pool_size=(14,14), strides=(14,14), border_mode='valid', name='spp2'))
model.add(Flatten())

# FC layers group
model.add(Dense(4096, activation='relu', name='fc6'))
model.add(Dropout(.5))
model.add(Dense(4096, activation='relu', name='fc7'))
model.add(Dropout(.5))
model.add(Dense(num_classes, activation='softmax', name='fc8'))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3), activation="relu", name="conv1", input_shape=(9, 30, 60..., strides=(1, 1, 1), padding="same", data_format="channels_last")`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(256, (3, 3, 3), activation="relu", name="conv2", strides=(1, 1, 1), padding="same", data_format="channels_last")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="pool2", padding="valid")`
/usr/local/lib/python3.6/dist-p

In [0]:
#To reload the model
from keras.models import load_model
from keras.optimizers import Adam

optimizer = Adam(lr=1e-5, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=10,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test))
#To save the model
model.save('newmodelA.h5')
del model
#To save model in google drive (If doesnt work run the lines on the top again)
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
f = drive.CreateFile({'title': 'newmodelA.h5'})
f.SetContentFile('newmodelA.h5')
f.Upload()


Train on 193118 samples, validate on 9831 samples
Epoch 1/2
193118/193118 [==============================] - 8706s 45ms/step - loss: 4.0281 - acc: 0.1095 - val_loss: 3.2431 - val_acc: 0.2259
Epoch 2/2
 61590/193118 [========>.....................] - ETA: 1:36:52 - loss: 3.1792 - acc: 0.2314

In [0]:
#To save model in google drive (If doesnt work run the lines on the top again)
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
f = drive.CreateFile({'title': 'point5modelA.h5'})
f.SetContentFile('point5modelA.h5')
f.Upload()

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download= drive.CreateFile({'id':'1rveBZNysQ-rtuG_x1MeY4HzEfZYkFNP-'})
download.GetContentFile('point5modelD.h5')
from keras.models import load_model
model = load_model('point5modelD.h5')
model.fit(X_train, y_train,
          batch_size=10,
          epochs=3,
          verbose=1,
          validation_data=(X_test, y_test))
#To save the model
model.save('point5modelE.h5')
del model
import os

#To save model in google drive (If doesnt work run the lines on the top again)
f = drive.CreateFile({'title': 'point5modelE.h5'})
f.SetContentFile('point5modelE.h5')
f.Upload()